In [7]:
!pip3 install python-statemachine
%run helpers.ipynb

In [10]:
from statemachine import StateMachine, State
import numpy as np

In [12]:
def b_dot(mag_field_vector, m, h, w, d):
    dt = 0.32
    T = 90 * 60 * 3
    t = np.arange(0, T + dt, dt)

    # Initializations
    inertia = inertia(m, h, w, d)

    alti = 600000  # in meters
    ang_vel = np.zeros((len(t), 3))
    cang = np.array([0.1, -1.5, 0.3])
    Re = 6371.2e3
    Rc = Re + alti
    p = ((111e3 * Rc) / Re)
    G = 6.67428e-11
    M = 5.972e24
    linvel = np.sqrt(G * M / Rc)
    angacc = np.array([0, 0, 0])

    # Main loop
    for i in range(len(t)):
        cangi = cang
        lati = (linvel * i * dt) / p
        lat = 25 + lati
        long = 60 + ((7.29e-5 * i * dt) * 180 / np.pi)

        if lat > 90:
            b = lat % 90
            lat = 90 - b
        if lat < -90:
            c = lat % -90
            lat = -90 - c
        if long > 180:
            d = long % 180
            long = d - 180
        if long < -180:
            l = long % -180
            long = 180 + l
        if lat == 90:
            lat = 89.97
        if lat == -90:
            lat = -89.97

        mag_field_vector1 = mag_field_vector
        mag_field_vector = mag_field_vector1 * 1e-9

        # B-dot
        detb = np.sqrt(np.dot(mag_field_vector, mag_field_vector))
        m = ((-1) / detb) * np.cross(mag_field_vector, cang)
        vtorque = np.cross(m, mag_field_vector)
        inertiain = np.linalg.inv(inertia)
        angacc = np.dot(vtorque, inertiain)
        cang = cangi + (angacc * dt)

        print('new angular velocity:')
        print(cang)
        ang_vel[i, :] = cang

NameError: name 'igrfmagm' is not defined

In [6]:
class ADCS(StateMachine):
    """
    This class functions as the ADCS Controller.
    
    inputs = []
    outputs = []
    states = []
    
    transitions
    -----------
    
    """
    
    off = State(initial=True)
    detumbling = State()
    pointing = State()
    error = State()
    
    on_initialization = off.to(detumbling)
    to_pointing = detumbling.to(pointing)
    continuous_pointing = pointing.to(pointing)
    
    def __init__(self):
        self.moment = None
        self.pointing_vec = [0, 0, 0]
        super(ADCS, self).__init__()
        
    def before_pointing(self):
        
        # slow down rotation
        
    def after_pointing(self):
        
        # update pointing vector
    
    def get_moment(self):
        return self.moment
    

class SafetyMonitor(StateMachine):
    
    safe = State(initial=True)
    error = State()
    
    to_error = safe.to(error, cond="moment_error")
    
    def moment_error(moment):
        return moment > 0.12
    
    def get_state(self):
        return self.current_state.id

IndentationError: expected an indented block (<ipython-input-6-f99f49d5ef8b>, line 97)